In [236]:
import pandas as pd 
import numpy as np 
import json as js
import csv
import matplotlib.pyplot as plt
import re
import os

In [2]:
%matplotlib inline

### Read Data

In [344]:
def read(df, output_columns, category, department, start_year=1):
    
    cohort_index = df.index[df.iloc[:, 0] == 'Headcount'][0]
    years_index = df.index[df.iloc[:, 0].map(lambda x: (True if re.search('^after.*(year|years)$', x, re.IGNORECASE) else False) if type(x) == str else False)].tolist()
    batch_years = df.iloc[cohort_index, :][~pd.isnull(df.iloc[cohort_index, :])][1:].map(lambda x: int(x)).tolist()
    batch_index = df.iloc[cohort_index, :][~pd.isnull(df.iloc[cohort_index, :])][1:].index.tolist()
    
    data = []

    def proc_data(info, i):
        for idx in range(len(batch_index)):
            batch = batch_years[idx]
            j = batch_index[idx]
            value = df.iloc[i, j]
            if not pd.isnull(value):
                row = [department, category, batch] + info
                row.append(value)
                data.append(row)

    def sweep_right(df, top, bottom, j, wall, info, callback, discard_first_column = True):
        if j == wall:
            if (bottom - top) != 1:
                raise Exception('sweep_right must arrive at a single cell at the rightmost column of the Headcount section.')
            callback(info, top)
            return

        this_top = None
        for i in range(top, bottom):
            if this_top and not pd.isnull(df.iloc[i, j]):
                if discard_first_column and j==1:
                    sweep_right(df, this_top, i, j+1, wall, info, callback)
                else:
                    sweep_right(df, this_top, i, j+1, wall, info + [df.iloc[this_top, j]], callback)
                this_top = i
                continue
            elif not pd.isnull(df.iloc[i, j]):
                this_top = i
                continue
            elif this_top:
                continue
            else:
                raise Exception('First row is empty!')
        if not re.search('.*total$', df.iloc[this_top, j], re.IGNORECASE):
            if discard_first_column and j==1:
                sweep_right(df, this_top, bottom, j+1, wall, info, callback)
            else:
                sweep_right(df, this_top, bottom, j+1, wall, info + [df.iloc[this_top, j]], callback)
    
    years = list(range(start_year, start_year+len(years_index)+1))

    for i in range(0, len(years_index)-1):
        top = years_index[i]
        bottom = years_index[i+1]
        sweep_right(df, top, bottom, 1, batch_index[0], [years[i]], proc_data)

    last_bottom = df.shape[0]
    for i in range(years_index[-1] + 1, df.shape[0]):
        if not pd.isnull(df.iloc[i, 0]):
            last_bottom = i

    sweep_right(df, years_index[-1], last_bottom, 1, batch_index[0], [years[-1]], proc_data)

    return pd.DataFrame(data, columns=output_columns)

In [345]:
folder_name = 'Major Flow Data'
create_data_params = []

for (dir_path, _, files) in list(os.walk(folder_name + '/'))[1:]:
    for file_name in files:
        prefix = re.search('.*(?=Flow by Major Progression Report.xlsx$)', file_name).group(0).strip()
        if '_' in prefix[:-1]:
            category = re.search('(?<=_).*', prefix[:-1]).group(0)
        else:
            category = 'All'
        department = re.search('^.*?(?=_)', prefix).group(0)
        create_data_params.append([dir_path + '/' + file_name, department, category])

In [346]:
df_continuing = []
df_graduated = []

for (path, department, category) in create_data_params:
    df = pd.read_excel(path, sheetname='Continuing Detail_2', header=None)
    col_names = ['department', 'category', 'batch', 'year', 'major_from', 'code_from', 'status', 'college_to', 'major_to', 'code_to', 'number']
    continuing = read(df, col_names, category=category, department=department, start_year=1)
    
    df = pd.read_excel(path, sheetname='Graduated Detail_3', header=None)
    col_names = ['department', 'category', 'batch', 'year', 'major_from', 'code_from', 'status', 'college_to', 'major_to', 'number']
    graduated = read(df, col_names, category=category, department=department, start_year=3)
    
    df_continuing.append(continuing)
    df_graduated.append(graduated)

In [347]:
continuing_combined = pd.concat(df_continuing, ignore_index=True)
graduated_combined = pd.concat(df_graduated, ignore_index=True)
graduated_combined['code_to'] = ''
continuing_combined['flow_type'] = 'Continuing'
graduated_combined['flow_type'] = 'Graduated'
data_combined = pd.concat((continuing_combined, graduated_combined), ignore_index=True)

In [350]:
data_combined.head()

,batch,category,code_from,code_to,college_to,department,flow_type,major_from,major_to,number,status,year
0,20049,All,E12200,E12200,NATURAL SCIENCES,Astronomy,Continuing,AST BS ASTRONOMY,AST BS ASTRONOMY,3.0,Continuing - Same Major,1
1,20059,All,E12200,E12200,NATURAL SCIENCES,Astronomy,Continuing,AST BS ASTRONOMY,AST BS ASTRONOMY,3.0,Continuing - Same Major,1
2,20069,All,E12200,E12100,NATURAL SCIENCES,Astronomy,Continuing,AST BS ASTRONOMY,ASTRONOMY ENTRY,1.0,Cont in Another Major - Same College,1
3,20049,All,E12200,E20900,NATURAL SCIENCES,Astronomy,Continuing,AST BS ASTRONOMY,CHEM (BSCH) CHEM,1.0,Cont in Another Major - Same College,1
4,20069,All,E12200,434500,COCKRELL SCHL OF ENGINEERING,Astronomy,Continuing,AST BS ASTRONOMY,ELECTRICAL ENGR,1.0,Cont - Another College,1


### Insert Data

In [348]:
import psycopg2

In [349]:
conn = psycopg2.connect("dbname=cnssankey user=xiya")
cur = conn.cursor()

cur.execute("DELETE FROM sankey_studentflow;")

for row in data_combined.as_matrix():
    cur.execute("INSERT INTO sankey_studentflow (" + ",".join(data_combined.columns.tolist()) + ") VALUES (%s" + ", %s" * (len(row)-1) + ");" , row.tolist())

conn.commit()
cur.close()
conn.close()